In [1]:
import warnings
import pandas as pd
from nltk.corpus import wordnet
import string,re
import nltk
import numpy as np
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer,LancasterStemmer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [26]:
warnings.filterwarnings("ignore")

In [2]:
#Read the hotel reviews dataset
hotel_reviews = pd.read_csv("Hotel_Reviews.csv")

In [3]:
hotel_reviews.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [4]:
#Select only relevant columns
Hotel_reviews = hotel_reviews[['Hotel_Name','Negative_Review','Positive_Review','Reviewer_Score','Average_Score']].copy()
Hotel_reviews.head()

,Hotel_Name,Negative_Review,Positive_Review,Reviewer_Score,Average_Score
0,Hotel Arena,I am so angry that i made this post available...,Only the park outside of the hotel was beauti...,2.9,7.7
1,Hotel Arena,No Negative,No real complaints the hotel was great great ...,7.5,7.7
2,Hotel Arena,Rooms are nice but for elderly a bit difficul...,Location was good and staff were ok It is cut...,7.1,7.7
3,Hotel Arena,My room was dirty and I was afraid to walk ba...,Great location in nice surroundings the bar a...,3.8,7.7
4,Hotel Arena,You When I booked with your company on line y...,Amazing location and building Romantic setting,6.7,7.7


In [5]:
##Concatenate the positive reviews and negative reviews
Hotel_reviews['reviews'] = Hotel_reviews[['Positive_Review', 'Negative_Review']].apply(lambda x: ' '.join(x), axis = 1)
Hotel_reviews.loc[2,'reviews']

' Location was good and staff were ok It is cute hotel the breakfast range is nice Will go back   Rooms are nice but for elderly a bit difficult as most rooms are two story with narrow steps So ask for single level Inside the rooms are very very basic just tea coffee and boiler and no bar empty fridge '

In [56]:
Hotel_reviews["is_good_review"] = Hotel_reviews["Reviewer_Score"].apply(lambda x: 1 if x > 5 else 0)
Hotel_reviews["is_bad_review"] = Hotel_reviews["Reviewer_Score"].apply(lambda x: 1 if x < 5 else 0)

In [57]:
Hotel_reviews.head()

,Hotel_Name,Negative_Review,Positive_Review,Reviewer_Score,Average_Score,reviews,is_good_review,is_bad_review,neg,neu,...,word_and,word_for,word_in,word_of,word_room,word_staff,word_the,word_to,word_very,word_was
0,Hotel Arena,I am so angry that i made this post available...,Only the park outside of the hotel was beauti...,2.9,7.7,Only the park outside of the hotel was beauti...,0,1,0.084,0.865,...,0.185212,0.069776,0.221178,0.239512,0.372173,0.000000,0.664287,0.406130,0.000000,0.331376
1,Hotel Arena,No Negative,No real complaints the hotel was great great ...,7.5,7.7,No real complaints the hotel was great great ...,1,0,0.058,0.757,...,0.606993,0.182942,0.165684,0.179417,0.000000,0.301843,0.480456,0.304231,0.320904,0.144803
2,Hotel Arena,Rooms are nice but for elderly a bit difficul...,Location was good and staff were ok It is cut...,7.1,7.7,Location was good and staff were ok It is cut...,1,0,0.122,0.740,...,0.530702,0.533162,0.000000,0.000000,0.000000,0.219921,0.350057,0.000000,0.467617,0.211004
3,Hotel Arena,My room was dirty and I was afraid to walk ba...,Great location in nice surroundings the bar a...,3.8,7.7,Great location in nice surroundings the bar a...,0,1,0.103,0.783,...,0.267059,0.000000,0.416549,0.338307,0.188709,0.047429,0.566213,0.334631,0.050424,0.409557
4,Hotel Arena,You When I booked with your company on line y...,Amazing location and building Romantic setting,6.7,7.7,Amazing location and building Romantic settin...,1,0,0.058,0.861,...,0.674405,0.254074,0.000000,0.249179,0.208490,0.104802,0.333634,0.000000,0.000000,0.502763


In [8]:
##Remove the "No negative" and "No positive" words from the reviews
##If no comment was left by user then "No negative" or "No positive" is written instead
Hotel_reviews["reviews"] = Hotel_reviews["reviews"].apply(lambda x: x.replace("No Negative", "").replace("No Positive", ""))

In [10]:
#Feature engineering
#Generate features that will be used in the models
#Features will be gotten from the sentiment analyzer, and TFIDF vectors
from nltk.sentiment.vader import SentimentIntensityAnalyzer

senti_analyser = SentimentIntensityAnalyzer()
Hotel_reviews["sentiments_reviews"] = Hotel_reviews["reviews"].apply(lambda x: senti_analyser.polarity_scores(x))
Hotel_reviews = pd.concat([Hotel_reviews.drop(['sentiments_reviews'], axis=1), Hotel_reviews['sentiments_reviews'].apply(pd.Series)], axis=1)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [58]:
Hotel_reviews.head()

,Hotel_Name,Negative_Review,Positive_Review,Reviewer_Score,Average_Score,reviews,is_good_review,is_bad_review,neg,neu,...,word_and,word_for,word_in,word_of,word_room,word_staff,word_the,word_to,word_very,word_was
0,Hotel Arena,I am so angry that i made this post available...,Only the park outside of the hotel was beauti...,2.9,7.7,Only the park outside of the hotel was beauti...,0,1,0.084,0.865,...,0.185212,0.069776,0.221178,0.239512,0.372173,0.000000,0.664287,0.406130,0.000000,0.331376
1,Hotel Arena,No Negative,No real complaints the hotel was great great ...,7.5,7.7,No real complaints the hotel was great great ...,1,0,0.058,0.757,...,0.606993,0.182942,0.165684,0.179417,0.000000,0.301843,0.480456,0.304231,0.320904,0.144803
2,Hotel Arena,Rooms are nice but for elderly a bit difficul...,Location was good and staff were ok It is cut...,7.1,7.7,Location was good and staff were ok It is cut...,1,0,0.122,0.740,...,0.530702,0.533162,0.000000,0.000000,0.000000,0.219921,0.350057,0.000000,0.467617,0.211004
3,Hotel Arena,My room was dirty and I was afraid to walk ba...,Great location in nice surroundings the bar a...,3.8,7.7,Great location in nice surroundings the bar a...,0,1,0.103,0.783,...,0.267059,0.000000,0.416549,0.338307,0.188709,0.047429,0.566213,0.334631,0.050424,0.409557
4,Hotel Arena,You When I booked with your company on line y...,Amazing location and building Romantic setting,6.7,7.7,Amazing location and building Romantic settin...,1,0,0.058,0.861,...,0.674405,0.254074,0.000000,0.249179,0.208490,0.104802,0.333634,0.000000,0.000000,0.502763


In [22]:
#calculate the tfidf score 
tfidf = TfidfVectorizer(max_features=10)
Hotel_reviews["tfid"] =  tfidf.fit_transform(Hotel_reviews['reviews']).toarray()


In [60]:
Hotel_reviews["tfid"].head()

0    0.185212
1    0.606993
2    0.530702
3    0.267059
4    0.674405
Name: tfid, dtype: float64

In [14]:
result = tfidf.fit_transform(Hotel_reviews['reviews']).toarray()
rev_tfidf = pd.DataFrame(result, columns = tfidf.get_feature_names())
rev_tfidf.columns = ["word_" + str(x) for x in rev_tfidf.columns]
rev_tfidf.index = Hotel_reviews.index
Hotel_reviews = pd.concat([Hotel_reviews, rev_tfidf], axis=1)

In [20]:
Hotel_reviews.head()

,Hotel_Name,Negative_Review,Positive_Review,Reviewer_Score,Average_Score,reviews,is_good_review,is_bad_review,neg,neu,...,word_and,word_for,word_in,word_of,word_room,word_staff,word_the,word_to,word_very,word_was
0,Hotel Arena,I am so angry that i made this post available...,Only the park outside of the hotel was beauti...,2.9,7.7,Only the park outside of the hotel was beauti...,0,0,0.084,0.865,...,0.185212,0.069776,0.221178,0.239512,0.372173,0.000000,0.664287,0.406130,0.000000,0.331376
1,Hotel Arena,No Negative,No real complaints the hotel was great great ...,7.5,7.7,No real complaints the hotel was great great ...,1,1,0.058,0.757,...,0.606993,0.182942,0.165684,0.179417,0.000000,0.301843,0.480456,0.304231,0.320904,0.144803
2,Hotel Arena,Rooms are nice but for elderly a bit difficul...,Location was good and staff were ok It is cut...,7.1,7.7,Location was good and staff were ok It is cut...,1,1,0.122,0.740,...,0.530702,0.533162,0.000000,0.000000,0.000000,0.219921,0.350057,0.000000,0.467617,0.211004
3,Hotel Arena,My room was dirty and I was afraid to walk ba...,Great location in nice surroundings the bar a...,3.8,7.7,Great location in nice surroundings the bar a...,0,0,0.103,0.783,...,0.267059,0.000000,0.416549,0.338307,0.188709,0.047429,0.566213,0.334631,0.050424,0.409557
4,Hotel Arena,You When I booked with your company on line y...,Amazing location and building Romantic setting,6.7,7.7,Amazing location and building Romantic settin...,1,1,0.058,0.861,...,0.674405,0.254074,0.000000,0.249179,0.208490,0.104802,0.333634,0.000000,0.000000,0.502763


In [23]:
label= ["is_good_review"]
columns = ["is_good_review","Hotel_Name","Negative_Review","Positive_Review","is_bad_review","Reviewer_Score","Average_Score","reviews","clean_reviews"]
features = [c for c in Hotel_reviews.columns if c not in columns]
features

['neg',
 'neu',
 'pos',
 'compound',
 'tfid',
 'word_and',
 'word_for',
 'word_in',
 'word_of',
 'word_room',
 'word_staff',
 'word_the',
 'word_to',
 'word_very',
 'word_was']

In [24]:
#Classification Algorithms
#Random Forest classifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(Hotel_reviews[features], Hotel_reviews[label], test_size = 0.20, random_state = 42)

random_forest = RandomForestClassifier(n_estimators = 100, random_state=42)
random_forest.fit(X_train, y_train)
y_pred = random_forest.predict(X_test)
#Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)
#Classification report
cf = classification_report(y_test, y_pred)
print(cf)
#Accuracy score
score = accuracy_score(y_test,y_pred)
print(score)

scores = cross_val_score(random_forest,X_train, y_train, cv=5)
scores.mean()

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.


[[  734  5494]
 [  771 96149]]
             precision    recall  f1-score   support

          0       0.49      0.12      0.19      6228
          1       0.95      0.99      0.97     96920

avg / total       0.92      0.94      0.92    103148

0.9392620312560592


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\P

0.9401245788797595

In [101]:
#Run the random forest model 
random_forest2 = RandomForestClassifier(n_estimators = 100, random_state=42)
random_forest2.fit(X_train, y_train)
y_pred = random_forest2.predict(X_test)
#Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)
#Classification report
cf = classification_report(y_test, y_pred)
print(cf)
#Accuracy score
score = accuracy_score(y_test,y_pred)
print(score)

scores = cross_val_score(random_forest,X_train, y_train, cv=5)
scores.mean()

[[  734  5494]
 [  771 96149]]
             precision    recall  f1-score   support

          0       0.49      0.12      0.19      6228
          1       0.95      0.99      0.97     96920

avg / total       0.92      0.94      0.92    103148

0.9392620312560592


0.9401245788797595

In [25]:
#SVM
cl = svm.SVC(kernel='linear')
cl.fit(X_train, y_train)
y_pred = cl.predict(X_test)
#Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)
#Classification report
cf = classification_report(y_test, y_pred)
print(cf)
#Accuracy score
score = accuracy_score(y_test,y_pred)
print(score)

scores = cross_val_score(cl,X_train, y_train, cv=5)
scores.mean()

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[    0  6228]
 [    0 96920]]


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.00      0.00      0.00      6228
          1       0.94      1.00      0.97     96920

avg / total       0.88      0.94      0.91    103148

0.9396207391321209


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-pac

0.9406796092973654

In [102]:
#Run the SVM model
cl2 = svm.SVC(kernel='linear')
cl2.fit(X_train, y_train)
y_pred = cl2.predict(X_test)
#Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)
#Classification report
cf = classification_report(y_test, y_pred)
print(cf)
#Accuracy score
score = accuracy_score(y_test,y_pred)
print(score)

scores = cross_val_score(cl,X_train, y_train, cv=5)
scores.mean()

[[    0  6228]
 [    0 96920]]
             precision    recall  f1-score   support

          0       0.00      0.00      0.00      6228
          1       0.94      1.00      0.97     96920

avg / total       0.88      0.94      0.91    103148

0.9396207391321209


0.9406796092973654

In [105]:
#Get the score showing the relevance of the features to the model
important_features = pd.DataFrame({"feature": features, "score": random_forest.feature_importances_}).sort_values("score", ascending = False)
important_features.head(15)

,feature,score
3,compound,0.189996
0,neg,0.108672
2,pos,0.104579
1,neu,0.101437
11,word_the,0.061405
9,word_room,0.051872
14,word_was,0.051318
5,word_and,0.047251
4,tfid,0.047145
12,word_to,0.046784


In [107]:
#Explarotary analysis
# add number of words column to count the number of words in reviews
Hotel_reviews["number_of_words"] = Hotel_reviews["reviews"].apply(lambda x: len(x.split(" ")))

In [109]:
#Highest positive reviews
Hotel_reviews[Hotel_reviews["number_of_words"] >= 10].sort_values("pos", ascending = False)[["reviews", "pos"]].head(10)

,reviews,pos
405823,Great hotel Wonderful staff Great value Ideal,0.898
304872,Perfect location Great Staff Excellent value ...,0.894
389173,Great location Clean safe comfortable friendl...,0.890
253599,Amazing Comfortable Great location Great bar ...,0.884
469147,Perfect room chic clean comfortable and bright,0.880
481531,Beautiful room courteous helpful staff great ...,0.873
339766,Excellent hotel great bars lovely helpful sta...,0.872
510272,Excellent staff wonderful room clean well equ...,0.872
128859,Clean tidy comfortable good value highly reco...,0.870
44728,Super clean and comfortable I liked everything,0.864


In [110]:
##Lowest negative reviews
Hotel_reviews[Hotel_reviews["number_of_words"] >= 10].sort_values("neg", ascending = False)[["reviews", "neg"]].head(10)

,reviews,neg
276520,Very poor service arrogant rude staff Poor s...,0.764
353800,poor facility poor staff poor management poor...,0.756
276763,terrrible unfriendly staff dirty ugly lobby ...,0.748
487716,Nothing Every thing was so bad Reseption bad...,0.737
136288,Nothing the stuff was bad very bad bad bad,0.737
266951,location terrible terrible breakfast really ...,0.726
396065,Terrible hotel Rude and poor staff Hate it ...,0.721
350172,Terrible terrible terrible and extortionate ...,0.718
384771,Very bad atmosphear noisy weird smells unfri...,0.713
287053,We have no complaints We have no complaints,0.710
